# Acelerando el gradient-boosting

En gradient-boosting, el algoritmo es secuencial: requiere que los árboles N-1 hayan sido ajustados para poder ajustar el árbol en la etapa N. 
- Por tanto, el algoritmo es bastante costoso. 
- La parte más cara es la búsqueda de la mejor división en el árbol, que es una aproximación de fuerza bruta: se evalúan todas las divisiones posibles y se elige la mejor.

Para acelerar el algoritmo, se puede reducir el número de divisiones a evaluar. 
- el rendimiento de generalización de dicho árbol se reducirá. 
- sin embargo, dado que estamos combinando varios árboles, podemos agregar más estimadores para superar este problema.

In [ ]:
# California housing dataset.

from sklearn.datasets import fetch_california_housing

data, target = fetch_california_housing(return_X_y=True, as_frame=True)
target *= 100

In [ ]:
# Haga un benchmark rápido del gradient-boosting original.

from sklearn.model_selection import cross_validate
from sklearn.ensemble import GradientBoostingRegressor

gradient_boosting = GradientBoostingRegressor(n_estimators=200)
cv_results_gbdt = cross_validate(
    gradient_boosting, data, target, scoring="neg_mean_absolute_error",
    n_jobs=2
)

In [ ]:
print("Gradient Boosting Decision Tree")
print(f"Error absoluto medio por validación cruzada: "
      f"{-cv_results_gbdt['test_score'].mean():.3f} ± "
      f"{cv_results_gbdt['test_score'].std():.3f} k$")
print(f"Tiempo de ajuste promedio: "
      f"{cv_results_gbdt['fit_time'].mean():.3f} seconds")
print(f"Tiempo de puntaje promedio: "
      f"{cv_results_gbdt['score_time'].mean():.3f} seconds")

- Recordemos que una forma de acelerar el gradient-boosting es reducir el número de divisiones consideradaas dentro de la agrupación de árboles.
- Una forma es agrupar los datos antes de darlo al gradient-boosting.
    - El transformador KBinsDiscretizer los hace.
    - Podemos generar un pipeline con este preprocesamiento.

In [ ]:
# vamos a demostrar la transformación realizada por el KbinsdIsCretizer.

import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(
    n_bins=256, encode="ordinal", strategy="quantile")
data_trans = discretizer.fit_transform(data)
data_trans

In [ ]:
# Cada valor representa el índice de bin cuando se procesa la distribución por cuantil. 
# Podemos verificar el número de contenedores por caraterística.

[len(np.unique(col)) for col in data_trans.T]

In [ ]:
# utilizaremos este transformador para discretizar los datos antes de entrenar al regresor de gradient boosting .

from sklearn.pipeline import make_pipeline

gradient_boosting = make_pipeline(
    discretizer, GradientBoostingRegressor(n_estimators=200))
cv_results_gbdt = cross_validate(
    gradient_boosting, data, target, scoring="neg_mean_absolute_error",
    n_jobs=2,
)

In [ ]:
print("Gradient Boosting Decision Tree with KBinsDiscretizer")
print(f"Error absoluto medio por validación cruzada: "
      f"{-cv_results_gbdt['test_score'].mean():.3f} ± "
      f"{cv_results_gbdt['test_score'].std():.3f} k$")
print(f"Tiempo de ajuste promedio: "
      f"{cv_results_gbdt['fit_time'].mean():.3f} seconds")
print(f"Tiempo de puntaje promedio: "
      f"{cv_results_gbdt['score_time'].mean():.3f} seconds")

- Scikit-Learn proporciona clases específicas que están aún más optimizadas para un conjunto de datos grande: `HistGradientBoostingClassifier` y `HistGradientBoostingRegressor`.
- Cada característica en los datos del dataset se agrupa primero al calcular los histogramas, luego se utilizan para evaluar las divisiones potenciales.
- El número de divisiones para evaluar es mucho más pequeño.
- Este algoritmo es mucho más eficiente que el gradient boosting cuando el conjunto de datos tiene más de 10,000 muestras.

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

histogram_gradient_boosting = HistGradientBoostingRegressor(
    max_iter=200, random_state=0)
cv_results_hgbdt = cross_validate(
    histogram_gradient_boosting, data, target,
    scoring="neg_mean_absolute_error", n_jobs=2,
)

In [ ]:
print("Árbol de decisión de impulso de gradiente de histograma")
print(f"Mean absolute error via cross-validation: "
      f"{-cv_results_hgbdt['test_score'].mean():.3f} ± "
      f"{cv_results_hgbdt['test_score'].std():.3f} k$")
print(f"Tiempo de ajuste promedio: "
      f"{cv_results_hgbdt['fit_time'].mean():.3f} seconds")
print(f"Tiempo de puntaje promedio: "
      f"{cv_results_hgbdt['score_time'].mean():.3f} seconds")

> - El histograma de gradient-boosting es el mejor algoritmo en términos de puntaje.
> - También escalará con el aumento de número de muestras, mientras que el gradient-boosting normal no lo hace.